In [1]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
from packages.load import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline


# 3. Shrinkage method:
## 3.1. Food:


In [2]:
hicp_all_path = 'data/preprocessed/hicp_yoy.csv'
hicp_class_path = 'data/HICP_COICOP10s.xlsx'
hicp_cat_path = 'data/preprocessed/food_yoy_infl.csv'
category = 'Food'

HICP_monthly, HICP_class, HICP_cat = import_data_all(hicp_all_path=hicp_all_path,
                                                     hicp_class_path=hicp_class_path,
                                                     hicp_cat_path=hicp_cat_path)

cat_df = split_into_category(category=category,
                             HICP_class=HICP_class,
                             HICP_monthly=HICP_monthly)

Number of items in Food group:  180


/Users/lananhnguyen/Desktop/thesis/thesis_code/main/packages/load.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df.fillna(0, inplace=True)


### Food using Ridge:

In [3]:
for i in [1, 2, 3]:
    X_cat_train, y_cat_train = split_train_set(cat_df=cat_df,
                                           HICP_cat=HICP_cat, 
                                           h = i)
        
    best_param = tuning_gridsearchcv(reg= Ridge(),
                                    grid_space= np.arange(0.9, 1, 0.01),
                                    X_train= X_cat_train,
                                    y_train= y_cat_train,
                                    test_size=12)
    print(best_param)
    print('----------')

Horizon: 1
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
{'regression__alpha': 0.9900000000000001}
----------
Horizon: 2
Training predictor period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Training dependent variable period: 1997-03-31 00:00:00 to 2015-12-31 00:00:00
{'regression__alpha': 0.9900000000000001}
----------
Horizon: 3
Training predictor period: 1997-01-31 00:00:00 to 2015-09-30 00:00:00
Training dependent variable period: 1997-04-30 00:00:00 to 2015-12-31 00:00:00
{'regression__alpha': 0.9900000000000001}
----------


In [4]:
ridge = RecursiveForecast(cat_df, HICP_cat, max_horizon= 3, model=Ridge, hyperparameter=1)
final_forecast_df = ridge.concat_forecast()

Train test split date: 2015-12-31, model <class 'sklearn.linear_model._ridge.Ridge'>, hyperparameter: 1
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Test predictor period: 2015-12-31 00:00:00 to 2022-11-30 00:00:00
Test dependent variable period: 2016-01-31 00:00:00 to 2023-07-31 00:00:00
--------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-11-30 00:00:00
Predictor test period: DatetimeIndex(['2015-11-30', '2015-12-31', '2016-01-31'], dtype='datetime64[ns]', name='date', freq=None)
Forecast: 
[ 2.38412775  1.66676772 -0.51997072]
-------------------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Predictor test period: DatetimeIndex(['2015-12-31', '

In [5]:
cat_file_path = 'data/forecast_results/food_forecast.csv'
save_forecast(forecast_result_df=final_forecast_df, cat_file_path=cat_file_path, category= 'food')

### Food using Lasso:

In [7]:
for i in [1, 2, 3]:
    X_cat_train, y_cat_train = split_train_set(cat_df=cat_df,
                                           HICP_cat=HICP_cat, 
                                           h = i)
        
    best_param = tuning_gridsearchcv(reg= Lasso(max_iter= 2000),
                                    grid_space= np.arange(0.01, 0.1, 0.01),
                                    X_train= X_cat_train,
                                    y_train= y_cat_train,
                                    test_size=12,
                                    )
    print(best_param)
    print('----------')

Horizon: 1
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
{'regression__alpha': 0.03}
----------
Horizon: 2
Training predictor period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Training dependent variable period: 1997-03-31 00:00:00 to 2015-12-31 00:00:00
{'regression__alpha': 0.06}
----------
Horizon: 3
Training predictor period: 1997-01-31 00:00:00 to 2015-09-30 00:00:00
Training dependent variable period: 1997-04-30 00:00:00 to 2015-12-31 00:00:00
{'regression__alpha': 0.06}
----------


In [9]:
lasso = RecursiveForecast(cat_df, HICP_cat, max_horizon= 3, model=Lasso, hyperparameter=0.06)
final_forecast_df = lasso.concat_forecast()

Train test split date: 2015-12-31, model <class 'sklearn.linear_model._coordinate_descent.Lasso'>, hyperparameter: 0.06
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Test predictor period: 2015-12-31 00:00:00 to 2022-11-30 00:00:00
Test dependent variable period: 2016-01-31 00:00:00 to 2023-07-31 00:00:00
--------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-11-30 00:00:00
Predictor test period: DatetimeIndex(['2015-11-30', '2015-12-31', '2016-01-31'], dtype='datetime64[ns]', name='date', freq=None)
Forecast: 
[2.26629333 2.09040043 1.64345521]
-------------------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Predictor test period: DatetimeIndex(['2

In [10]:
final_forecast_df

,Lasso_h_1,Lasso_h_2,Lasso_h_3
0,1.461044,1.502186,1.643455
1,1.559837,1.623757,1.712442
2,1.585073,1.608705,1.726576
3,1.326018,1.464185,1.486541
4,1.168614,1.288851,1.375892
...,...,...,...
79,13.175836,12.575702,12.07952
80,14.996388,14.174641,13.488595
81,15.974193,15.691168,14.856713
82,15.965465,15.798487,15.527859


In [ ]:
cat_file_path = 'data/forecast_results/food_forecast.csv'
save_forecast(forecast_result_df=final_forecast_df, cat_file_path=cat_file_path, category= 'food')

---

## 3.2. Energy:


In [2]:
hicp_all_path = 'data/preprocessed/hicp_yoy.csv'
hicp_class_path = 'data/HICP_COICOP10s.xlsx'
hicp_cat_path = 'data/preprocessed/energy_yoy_infl.csv'
category = 'Energy'

HICP_monthly, HICP_class, HICP_cat = import_data_all(hicp_all_path=hicp_all_path,
                                                     hicp_class_path=hicp_class_path,
                                                     hicp_cat_path=hicp_cat_path)

cat_df = split_into_category(category=category,
                             HICP_class=HICP_class,
                             HICP_monthly=HICP_monthly)
                             
X_cat_train, y_cat_train = split_train_set(cat_df=cat_df,
                                           HICP_cat=HICP_cat)

Number of items in Energy group:  14
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00


/Users/lananhnguyen/Desktop/thesis/thesis_code/main/packages/load.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df.fillna(0, inplace=True)


In [4]:

best_param = tuning_gridsearchcv(reg= Ridge(),
                                 grid_space= np.arange(0.1, 1, 0.01),
                                 X_train= X_cat_train,
                                 y_train= y_cat_train,
                                 test_size=12)
best_param

{'regression__alpha': 0.9899999999999995}

In [5]:
ridge = RecursiveForecast(cat_df, HICP_cat, max_horizon= 3, model=Ridge, hyperparameter=1)
final_forecast_df = ridge.concat_forecast()

Train test split date: 2015-12-31, model <class 'sklearn.linear_model._ridge.Ridge'>, hyperparameter: 1
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Test predictor period: 2015-12-31 00:00:00 to 2022-11-30 00:00:00
Test dependent variable period: 2016-01-31 00:00:00 to 2023-07-31 00:00:00
--------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-11-30 00:00:00
Predictor test period: DatetimeIndex(['2015-11-30', '2015-12-31', '2016-01-31'], dtype='datetime64[ns]', name='date', freq=None)
Forecast: 
[-8.63227017 -7.87524143 -7.3701297 ]
-------------------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Predictor test period: DatetimeIndex(['2015-12-31', '

In [6]:
cat_file_path = 'data/forecast_results/energy_forecast.csv'
save_forecast(forecast_result_df=final_forecast_df, cat_file_path=cat_file_path, category='energy')

CSV file 'data/forecast_results/energy_forecast.csv' created and DataFrame saved.
